# Classification Tasks

## BACH

In [5]:
from torchvision.datasets import ImageFolder, MNIST
from pathlib import Path
from torch.utils.data import random_split
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data import DataLoader
import numpy as np
from numpy.random import default_rng
from torchvision import transforms
import torch
from torchvision.datasets.utils import download_url, download_and_extract_archive
from torch.utils.data import Dataset

In [73]:
p = Path('/home/niklas/Internal_HDD/project_data/histopathology/BACH/ICIAR2018_BACH_Challenge/Photos')

In [3]:
mean, std = torch.tensor([0.7169, 0.6170, 0.8427]), torch.tensor([0.1661, 0.1885, 0.1182]) # calculated over dataset

In [74]:
class Bach:
    
    def __init__(self, root, train_transform, valid_transform, download=False, valid_percent=0.2, shuffle=True):
        
        self.root = root
        self.train_transform = train_transform
        self.valid_transform = valid_transform
        self.valid_percent = valid_percent
        self.shuffle = shuffle
        
        if download:
            self.download_data()
            self.root = self.root/'ICIAR2018_BACH_Challenge/Photos' # extend root directory to point to images
            
        self.train_ds, self.valid_ds, self.train_sampler, self.valid_sampler = self.prepare_datasets()
        
    def download_data(self):
        url = 'https://zenodo.org/record/3632035/files/ICIAR2018_BACH_Challenge.zip'
        download_and_extract_archive(url, self.root)
    
    def prepare_datasets(self):
        train_ds = ImageFolder(self.root, self.train_transform)
        valid_ds = ImageFolder(self.root, self.valid_transform)
        
        num_train = len(train_ds)
        indices   = list(range(num_train))
        split     = int(np.floor(self.valid_percent * num_train))
        
        if self.shuffle:
            rng = default_rng(seed=101)
            rng.shuffle(indices)
        
        train_idx, valid_idx = indices[split:], indices[:split]
        train_sampler = SubsetRandomSampler(train_idx)
        valid_sampler = SubsetRandomSampler(valid_idx)
        
        return train_ds, valid_ds, train_sampler, valid_sampler
    
    def get_dataloaders(self, batch_size, shuffle=True, pin_memory=True, num_workers=0):
        train_dl = DataLoader(
            self.train_ds, batch_size=batch_size, sampler=self.train_sampler,
            num_workers=num_workers, pin_memory=pin_memory
        )
        
        valid_dl = DataLoader(
            self.valid_ds, batch_size=batch_size, sampler=self.train_sampler,
            num_workers=num_workers, pin_memory=pin_memory
        )
        
        return train_dl, valid_dl

In [75]:
train_transform = transforms.Compose([transforms.RandomRotation(30),
                                       transforms.RandomResizedCrop(224),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.ToTensor()])

valid_transform = transforms.Compose([transforms.Resize(255),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor()])

In [76]:
bach = Bach(p, train_transform, valid_transform)

In [77]:
train_dl, valid_dl = bach.get_dataloaders(32)

In [79]:
x,y = next(iter(train_dl))

In [127]:
def get_mean_std(loader):
    # var[X] = E[X**2] - E[X]**2
    channels_sum, channels_sqrd_sum, num_batches = 0, 0, 0

    for data, _ in loader:
        channels_sum += torch.mean(data, dim=[0, 2, 3])
        channels_sqrd_sum += torch.mean(data ** 2, dim=[0, 2, 3])
        num_batches += 1

    mean = channels_sum / num_batches
    std = (channels_sqrd_sum / num_batches - mean ** 2) ** 0.5

    return mean, std


#mean, std = get_mean_std(dl)
#print(mean)
#print(std)

## Patchcamylion

In [52]:
class PatchCamelyonDataset(Dataset):

    def __init__(self, root, transform, mode='train'):
        super().__init__()

        assert mode in ['train', 'valid', 'test']
        
        self.root = root
        self.transform = transform
        self.mode = mode

        self.X = h5py.File(root/f'camelyonpatch_level_2_split_{mode}_x.h5', 'r').get('x')
        self.y = h5py.File(root/f'camelyonpatch_level_2_split_{mode}_y.h5', 'r').get('y')

    def __getitem__(self, idx):
        x, y = self.X[idx], self.y[idx]
        x, y = self.transform(x), y.item()
        return x, y

    def __len__(self):
        return len(self.X)

In [115]:
class PatchCamelyon:
    
    def __init__(self, root, train_transform, valid_transform, download=False):
        
        self.root = root
        self.train_transform = train_transform
        self.valid_transform = valid_transform
        
        if download:
            self.download_data()
            
        self.train_ds, self.valid_ds, self.test_ds = self.prepare_datasets()
        
    def download_data(self):
        base_url = 'https://zenodo.org/record/2546921/files/'
        for mode in ['train', 'valid', 'test']:
            download_url(base_url + f'camelyonpatch_level_2_split_{mode}_meta.csv', self.root)
            for xy in ['x','y']: 
                download_and_extract_archive(base_url + f'camelyonpatch_level_2_split_{mode}_{xy}.h5.gz', self.root)
    
    def prepare_datasets(self):
        train_ds = PatchCamelyonDataset(self.root, transform=self.train_transform, mode='train')
        valid_ds = PatchCamelyonDataset(self.root, transform=self.valid_transform, mode='valid')
        test_ds  = PatchCamelyonDataset(self.root, transform=self.valid_transform, mode='test')
        
        return train_ds, valid_ds, test_ds
    
    def get_dataloaders(self, batch_size, shuffle=True, pin_memory=True, num_workers=0):
        
        train_dl = DataLoader(
            self.train_ds, batch_size=batch_size, shuffle=shuffle,
            num_workers=num_workers, pin_memory=pin_memory)
        
        valid_dl = DataLoader(
            self.valid_ds, batch_size=batch_size,
            num_workers=num_workers, pin_memory=pin_memory)
        
        test_dl = DataLoader(
            self.test_ds, batch_size=batch_size,
            num_workers=num_workers, pin_memory=pin_memory)
        
        return train_dl, valid_dl, test_dl

In [116]:
tsfm = transforms.Compose([
    transforms.ToTensor()
])

In [117]:
root = Path('/home/niklas/Internal_HDD/project_data/histopathology/pcam/')

In [118]:
pcam = PatchCamelyon(root, train_transform=tsfm, valid_transform=tsfm, download=False)

In [129]:
train_dl, valid_dl, test_dl = pcam.get_dataloaders(128)

In [125]:
x,y = next(iter(test_dl))

In [124]:
x.shape

torch.Size([32, 3, 96, 96])

In [131]:
mean, std = get_mean_std(train_dl)
print(mean)
print(std)

tensor([0.7008, 0.5384, 0.6916])
tensor([0.2350, 0.2774, 0.2129])
